In [ ]:
!pip install --upgrade pip
!pip install gym
import gym
import numpy as np
from tensorflow import keras

# Define the Deep Q-Network (DQN) class
class DQNAgent:
    def __init__(self, state_shape, action_space):
        self.state_shape = state_shape
        self.action_space = action_space
        self.gamma = 0.99  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_decay = 0.995  # Rate at which exploration rate decays
        self.epsilon_min = 0.01  # Minimum exploration rate
        self.learning_rate = 0.001  # Learning rate

        self.model = self.build_model()

    def build_model(self):
        model = keras.Sequential()
        model.add(keras.layers.Conv2D(32, kernel_size=(8, 8), strides=(4, 4), activation='relu', input_shape=self.state_shape))
        model.add(keras.layers.Conv2D(64, kernel_size=(4, 4), strides=(2, 2), activation='relu'))
        model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
        model.add(keras.layers.Flatten())
        model.add(keras.layers.Dense(512, activation='relu'))
        model.add(keras.layers.Dense(self.action_space, activation='linear'))
        model.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=self.learning_rate))
        return model

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.randint(self.action_space)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def train(self, state, action, reward, next_state, done):
        target = reward
        if not done:
            target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        target_f = self.model.predict(state)
        target_f[0][action] = target
        self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Initialize the environment
env = gym.make('Pong-v0')
#env = gym.make('Breakout-v0')

# Extract state shape and action space information
state_shape = env.observation_space.shape
action_space = env.action_space.n

# Initialize the DQN agent
agent = DQNAgent(state_shape, action_space)

# Training parameters
num_games = 5
max_steps = 10000

# Training loop
for game in range(num_games):
    state = env.reset()
    state = np.reshape(state, [1] + list(state_shape))

    for step in range(max_steps):
        env.render()

        # Agent selects an action
        action = agent.act(state)

        # Perform the action and observe the next state, reward, and done flag
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1] + list(state_shape))

        # Reward shaping
        if reward == 1:  # If player wins a point
            reward = 10
        elif reward == -1:  # If opponent wins a point
            reward = -10
        else:
            reward


/usr/local/lib/python3.10/dist-packages/jaxlib/xla_client.py:225: DeprecationWarning: ml_dtypes.float8_e4m3b11 is deprecated. Use ml_dtypes.float8_e4m3b11fnuz
  float8_e4m3b11fnuz = ml_dtypes.float8_e4m3b11


NameNotFound: ignored